# 1 全概率公式

In [ ]:
%matplotlib inline

# matplotlib 对中文的支持及保存为 pdf 格式
from matplotlib import rcParams
rcParams['font.family'] = 'Microsoft YaHei'
rcParams["pdf.fonttype"] = 42

# Look pretty...
from matplotlib import style
style.use('ggplot')

# 设置 numpy 的输出精度, 并且阻止使用科学记数法，
# formatter 参数允许为每个类型指定一个格式化函数
np.set_printoptions( precision=3, suppress=True )

### 例 1

某同学每周上一次概率课，每次上完课后，他可能跟的上，也可能跟不上. 如果某一周的课他能跟的上，那么下一周的课他跟的上的概率为 0.8；如果跟不上，那么下一周的课他跟的上的概率为 0.3. 现在假定第一周上课前他能够跟的上课程，那么经过 n 周的学习，他能够跟上的概率有多大?

In [ ]:
####################
# 方法1: 递归方式
####################

def masterProb(n, prob1, prob2, initial_prob):
    """    
    n: 第 n 次课
    prob1: 在前一次课跟的上的情况下，第 n 次课跟的上的概率
    prob2: 在前一次课跟不上的情况下，第 n 次课跟的上的概率
    initial_prob: 第一次上课前跟的上的概率
    """
    if n == 1:
        return prob1 * initial_prob + prob2 * (1 - initial_prob)
    else:
        return prob1 * masterProb(n-1, prob1, prob2, initial_prob) + prob2 * (1 - masterProb(n-1, prob1, prob2, initial_prob))

In [ ]:
for n in range(1,20):
    print('第 {0} 次课后跟的上的概率为 {1:.5f}.'.format(n, masterProb(n, 0.8, 0.3, 1)))

递归程序的运行效率很低，因此我们采用下面第二种方法：概率转移矩阵. 令 $A_i$ 表示第 $i$ 次课后跟的上. 那么

$$
\left(\begin{array}{c}
P(A_{i+1}) \\
P(\bar{A}_{i+1})
\end{array}\right) = 
\left(\begin{array}{c}
P(A_{i+1}|A_i) & P(A_{i+1}|\bar{A}_i) \\
P(\bar{A}_{i+1}|A_i) & P(\bar{A}_{i+1}|\bar{A}_i)
\end{array}\right)
\left(\begin{array}{c}
P(A_{i}) \\
P(\bar{A}_{i})
\end{array}\right)
$$

其中 $\left(\begin{array}{c}
P(A_{i+1}|A_i) & P(A_{i+1}|\bar{A}_i) \\
P(\bar{A}_{i+1}|A_i) & P(\bar{A}_{i+1}|\bar{A}_i)
\end{array}\right) = \left(\begin{array}{c}
0.8 & 0.4 \\
0.2 & 0.6
\end{array}\right)$, $\left(\begin{array}{c}
P(A_{0}) \\
P(\bar{A}_{0})
\end{array}\right) = \left(\begin{array}{c}
1 \\
0
\end{array}\right)$.

In [ ]:
####################
# 方法2: 概率转移矩阵
####################

import numpy as np

# 初始概率分布，第一次课前跟得上的概率分布
# 用一个二维列向量表示，第一个元素表示跟得上的概率，
# 第二个元素表示跟不上的元素
initial_dist = np.array([1,0])

# 概率转移矩阵，表示第 i+1 次课能否听懂和第 i 次课的关系
# 用一个 2 * 2 矩阵表示，其中第 (i, j) 个元素表示在第 i 行的结果发生的情况下第 j 列的结果发生的概率
tranfer_matrix = np.array([[0.8,0.3],[0.2,0.7]])

# 经过 n 次后的结果，二维向量的第一个元素表示跟得上的概率，
# 第二个元素表示跟不上的元素
next_dist = initial_dist
for i in range(1, 30):    
    next_dist = np.dot(tranfer_matrix, next_dist)
    print('第 {0} 次课后跟的上的概率为 {1:.5f}.'.format(i, next_dist[0]))

### 例 2 三门问题

美国电视节目中经常出现如下情形：你站在 3 个封闭的门前，其中一个门后有大奖. 奖品在哪个门后是完全随机的. 当你选定一个门后，主持人打开其余两扇门中的一扇空门，然后说：你还有一次重新选择的机会，是否要改主意选另一扇没有被打开的门. 你会如何选择？
- 坚持原来的选择.
- 改选另一扇没有被打开的门.
- 无所谓，改变不改变没有影响.

In [ ]:
import pylab
import random

def openEmptyDoor(guess_door, prize_door):
    """
    定义主持人打开一扇空门的方法
    
    在奖品已经放在某个门的后面，且选手已经做出选择的情况下。
    若选手猜的不是放奖品的门，在打开另一扇空门。
    若选手猜的是放奖品的门，则在两扇空门中随机打开一扇。
    """
    if guess_door == prize_door:
        randomChoose(prize_door)
    else:
        if 1 != guess_door and 1 != prize_door:
            return 1
        if 2 != guess_door and 2 != prize_door:
            return 2
        return 3

def randomChoose(door):
    """
    从给定号码以为的号码中随机选一个
    """
    if door == 1:
        return random.choice([2,3])
    if door == 2:
        return random.choice([1,3])
    return random.choice([1,2])

def simMontyHall(num_trials):
    
    # 记录不改变决定取胜的次数和改变决定取胜的次数
    stick_wins, switch_wins = (0, 0)

    for t in range(num_trials):
        # 随机放置奖品
        prize_door = random.choice([1, 2, 3])
        
        # 随机猜一个门
        guess_door = random.choice([1, 2, 3])
        
        # 主持人打开一扇门
        open_empty_door = openEmptyDoor(guess_door, prize_door)
        
        if guess_door == prize_door:  # 不改变决定
            stick_wins += 1
        else:  # 当你猜的和奖品所在的门不一样时，只有改变决定才能获奖
            switch_wins += 1
    return (stick_wins, switch_wins)


def displayMHSim(sim_results, title):
    stick_wins, switch_wins = sim_results
    pylab.pie([stick_wins, switch_wins],
              colors = ['r', 'c'],
              labels = ['保持不变', '改变决定'],
              autopct = '%.2f%%')
    pylab.title(title)

sim_results = simMontyHall(100000)
displayMHSim(sim_results, 'Monty Hall 问题模拟')
pylab.figure()

# 2 贝叶斯公式

### 例 猜骰子游戏

我们有三枚公平的骰子，一枚正六面的、一枚正八面的、一枚正十二面的.
我们随机从中抽一枚，请你猜我抽出的是哪一枚？我通过不断告诉你我用
抽出的骰子进行投掷结果.

In [1]:
import numpy as np
import pandas as pd

def update(points, priori_dist, conditional_matrix):
    """
    输入先验分布和新信息，输出后验分布
    
    points: 掷得的点数
    priori_dist: 抽到各个骰子的概率的先验分布，是一个 np 数组.
    conditional_matrix: 在抽到某个骰子的情况下，掷出各个点的概率，
            存储为 pd.DataFrame，每一列存储一个骰子掷出各个点的概率
            每一行表示用各个骰子掷出当前行的点数的概率
    """
    
    # 读取各个骰子掷出 dice_points 的条件概率，需要转化为 np 对象
    conditional_array = conditional_matrix.loc[points].values
    
    # 根据全概率公式计算在给定抽到各个骰子的概率先验分布下，掷出当前点数的概率
    points_prob = np.dot(priori_dist, conditional_array)
    
    # 更新后验分布
    return (priori_dist * conditional_array) / points_prob

In [2]:
priori_dist = np.array([1/3, 1/3, 1/3])

conditional_matrix = pd.DataFrame({
    'dice6': [1/6,1/6,1/6,1/6,1/6,1/6,0,0,0,0,0,0], 
    'dice8': [1/8,1/8,1/8,1/8,1/8,1/8,1/8,1/8,0,0,0,0],
    'dice12': [1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12]},
    index=[str(i) for i in range(1, 13)])

conditional_matrix.head()

,dice6,dice8,dice12
1,0.166667,0.125,0.083333
2,0.166667,0.125,0.083333
3,0.166667,0.125,0.083333
4,0.166667,0.125,0.083333
5,0.166667,0.125,0.083333


In [ ]:
posterior_dist = priori_dist

points = '0'

print('输入任意字母，程序退出。')
while(points.isdigit()):
    points = input('请输入掷出的点数：')
    
    # 判断输入的点数是否在正确范围，不在范围直接退出
    if not (points.isdigit() and 0 < int(points) < 13):
        print('请输入正确的点数')
        continue
        
    posterior_dist = update(points, posterior_dist, conditional_matrix)
    print('各个骰子掷出 {} 点的可能性分别为 {}.'.format(points, str(posterior_dist)))

输入任意字母，程序退出。
请输入掷出的点数：3
各个骰子掷出 3 点的可能性分别为 [0.44444444 0.33333333 0.22222222].
请输入掷出的点数：15
请输入正确的点数
请输入掷出的点数：12
各个骰子掷出 12 点的可能性分别为 [0. 0. 1.].
请输入掷出的点数：3
各个骰子掷出 3 点的可能性分别为 [0. 0. 1.].
请输入掷出的点数：0
请输入正确的点数
请输入掷出的点数：6
各个骰子掷出 6 点的可能性分别为 [0. 0. 1.].
